In [1]:
# git config --global credential.helper store

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import itertools
from collections import Counter
import ssl
import time
import string
import unicodedata

from urllib.request import Request, urlopen
from threading import Thread
from bs4 import BeautifulSoup

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, RandomForestRegressor,
                            GradientBoostingClassifier, GradientBoostingRegressor)

import nltk
nltk.download(["stopwords", "punkt", "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"])
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import RegexpParser
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

from clean_dfs import clean_features, clean_weeks, clean_lyrics
from web_scraping import parse_page, store_lyrics
from nlp_pipeline import lyrics_tokenize, get_tfidf_matrix
from genre_helper_functions import get_bucket, contains_genre_type, create_genre_column
from make_plots import (make_frequency_plot, make_line_plot, make_dual_plot_same,
                        make_dual_plot_mixed, make_scatter)
import modeling_functions as mf

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]    

In [2]:
features = clean_features()
#weeks = clean_weeks()

In [3]:
'''joined = weeks.merge(features, on='SongID')
#joined.to_csv("data/joined.csv", index=False)

# Expand genres into individual components
featureGenres = features.explode('spotify_genre')
featureGenres = featureGenres[featureGenres['spotify_genre'] != '']

joinedGenres = joined.explode('spotify_genre')
joinedGenres = joinedGenres[joinedGenres['spotify_genre'] != '']

explicitness = joined[['Year', 'spotify_track_explicit']]
explicitness = explicitness.groupby(['Year']).mean().reset_index()

numericalMetrics = joined.columns.tolist()[11:23]
numericals = joined[['Year'] + numericalMetrics].groupby(['Year']).mean().reset_index()'''

'joined = weeks.merge(features, on=\'SongID\')\n#joined.to_csv("data/joined.csv", index=False)\n\n# Expand genres into individual components\nfeatureGenres = features.explode(\'spotify_genre\')\nfeatureGenres = featureGenres[featureGenres[\'spotify_genre\'] != \'\']\n\njoinedGenres = joined.explode(\'spotify_genre\')\njoinedGenres = joinedGenres[joinedGenres[\'spotify_genre\'] != \'\']\n\nexplicitness = joined[[\'Year\', \'spotify_track_explicit\']]\nexplicitness = explicitness.groupby([\'Year\']).mean().reset_index()\n\nnumericalMetrics = joined.columns.tolist()[11:23]\nnumericals = joined[[\'Year\'] + numericalMetrics].groupby([\'Year\']).mean().reset_index()'

In [4]:
'''# Normalize numerical features not between 0 and 1
featureGenresNorm = featureGenres.copy()
scaled = ["track_duration", "loudness", "tempo"]
for metric in scaled:
    mms = MinMaxScaler()
    featureGenresNorm[metric] = mms.fit_transform(featureGenresNorm['track_duration']. \
                                to_numpy().reshape(-1, 1))

# Create grouped tables
genres = featureGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoined = joinedGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoinedDecade = joinedGenres.groupby(['spotify_genre', 'Decade'])['SongID'].count(). \
                        reset_index().sort_values(by="Decade")
genreFeatures = featureGenresNorm.groupby(['spotify_genre'])[numericalMetrics].mean().reset_index()'''

'# Normalize numerical features not between 0 and 1\nfeatureGenresNorm = featureGenres.copy()\nscaled = ["track_duration", "loudness", "tempo"]\nfor metric in scaled:\n    mms = MinMaxScaler()\n    featureGenresNorm[metric] = mms.fit_transform(featureGenresNorm[\'track_duration\'].                                 to_numpy().reshape(-1, 1))\n\n# Create grouped tables\ngenres = featureGenres.groupby([\'spotify_genre\'])[\'SongID\'].count().reset_index()\ngenresJoined = joinedGenres.groupby([\'spotify_genre\'])[\'SongID\'].count().reset_index()\ngenresJoinedDecade = joinedGenres.groupby([\'spotify_genre\', \'Decade\'])[\'SongID\'].count().                         reset_index().sort_values(by="Decade")\ngenreFeatures = featureGenresNorm.groupby([\'spotify_genre\'])[numericalMetrics].mean().reset_index()'

In [5]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [6]:
'''# Web scrape lyrics
featureScrape = features.loc[[contains_genre_type(genre, ["pop", "rock", "metal"]) for genre \
                             in features['spotify_genre']]].reset_index(drop=True)
lyricsMap = {}
threads = []
temp = 0
start = time.time()
# Write scraped lyrics to hashmap, parallelize to save time (thread safe because no unique keys)
#for i in range(temp, temp+50):
for i in range(len(featureScrape)):
    t = Thread(target=store_lyrics, args=(featureScrape['Song'][i],
                featureScrape['Performer'][i], lyricsMap))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
end = time.time()
print(end - start)
scrapedLyrics = pd.DataFrame(lyricsMap.items(), columns=["SongID", "Lyrics"])
scrapedLyrics.to_csv("data/scrapedLyrics.csv", index=False)

# Get list of all improperly formatted songs and save to file
problemSongs = []
for k, v in lyricsMap.items():
    if v[0][0] == "*":
        problemSongs.append([k] + v[2:5])
print(len(featureScrape), len(problemSongs))

with open("data/problemSongs.txt", "w") as file:
    for s in problemSongs:
        file.write("{}\n".format(s))'''

'# Web scrape lyrics\nfeatureScrape = features.loc[[contains_genre_type(genre, ["pop", "rock", "metal"]) for genre                              in features[\'spotify_genre\']]].reset_index(drop=True)\nlyricsMap = {}\nthreads = []\ntemp = 0\nstart = time.time()\n# Write scraped lyrics to hashmap, parallelize to save time (thread safe because no unique keys)\n#for i in range(temp, temp+50):\nfor i in range(len(featureScrape)):\n    t = Thread(target=store_lyrics, args=(featureScrape[\'Song\'][i],\n                featureScrape[\'Performer\'][i], lyricsMap))\n    threads.append(t)\n    t.start()\nfor t in threads:\n    t.join()\nend = time.time()\nprint(end - start)\nscrapedLyrics = pd.DataFrame(lyricsMap.items(), columns=["SongID", "Lyrics"])\nscrapedLyrics.to_csv("data/scrapedLyrics.csv", index=False)\n\n# Get list of all improperly formatted songs and save to file\nproblemSongs = []\nfor k, v in lyricsMap.items():\n    if v[0][0] == "*":\n        problemSongs.append([k] + v[2:5])\nprin

In [7]:
# Read csv of previously outputted scraped lyrics and reformat to match original
allLyrics = clean_lyrics()

In [8]:
'''# NLP pipeline to create tokens from lyrics
allLyrics['Lyrics_tokenized'] = list(map(lyrics_tokenize, allLyrics['Lyrics']))
allLyrics.dropna(inplace=True)
allLyrics.to_csv("data/lyricsTokenized.csv", index=False)'''

'# NLP pipeline to create tokens from lyrics\nallLyrics[\'Lyrics_tokenized\'] = list(map(lyrics_tokenize, allLyrics[\'Lyrics\']))\nallLyrics.dropna(inplace=True)\nallLyrics.to_csv("data/lyricsTokenized.csv", index=False)'

In [9]:
###################

In [10]:
# Create corpus and make dataframe with TF-IDF matrix
allLyrics = pd.read_csv("data/lyricsTokenized.csv")
allLyrics.dropna(inplace=True)
corpus = allLyrics['Lyrics_tokenized']
tfidfLyrics = get_tfidf_matrix(corpus)
tfidfLyrics.insert(0, "SongID", allLyrics['SongID'])
tfidfLyrics.to_csv("data/tfidfMatrix.csv")

In [11]:
tfidfLyrics = pd.read_csv("data/tfidfMatrix.csv", index_col=0)


69.72286880679317

6
33
35
41
58
83
91
92
106
112
140
162
186
192
193
195
198
234
235
237
252
262
269
290
296
319
335
339
345
355
397
458
460
477
486
496
565
574
618
643
644
651
675
697
710
740
747
815
834
863
924
975
987
995
1007
1017
1098
1106
1113
1114
1116
1145
1150
1269
1275
1278
1350
1354
1377
1396
1415
1419
1430
1436
1440
1446
1455
1459
1481
1502
1535
1564
1605
1668
1712
1719
1729
1770
1807
1814
1818
1833
1839
1860
1868
1876
1896
1898
1907
1922
1953
1961
1983
1992
1994
2018
2050
2062
2072
2150
2153
2160
2190
2193
2196
2258
2304
2347
2350
2378
2406
2415
2416
2439
2441
2457
2468
2490
2507
2515
2539
2553
2556
2567
2568
2587
2596
2627
2628
2633
2712
2737
2752
2767
2776
2791
2801
2874
2939
2967
2973
3025
3028
3058
3066
3068
3098
3104
3113
3122
3148
3149
3153
3199
3228
3263
3284
3306
3342
3352
3353
3355
3362
3380
3406
3453
3511
3610
3620
3687
3688
3726
3731
3735
3803
3811
3817
3840
3853
3861
3889
3936
3963
3979
3985
3989
4019
4027
4030
4047
4055
4062
4070
4105
4110
4117
4120
4133
4157
4163
4195
4208
425

In [14]:
model = Sequential()